## Data Processing of Mcdonald's restaurants reviews

### Import utilities

In [57]:
import pandas as pd
import zipfile
import re

from dateutil.relativedelta import relativedelta

### Extract dataset from Kaggle

In [ ]:
! pip install -q kaggle
! pip install -q kaggle-cli

In [65]:
# Set access to Kaggle API
! mkdir -p ~/.kaggle
! cp "/teamspace/studios/this_studio/PROJECT_FINAL/.kaggle/kaggle.json" ~/.kaggle/
! cat ~/.kaggle/kaggle.json

! chmod 600 /teamspace/studios/this_studio/PROJECT_FINAL/.kaggle/kaggle.json

{"username":"yonapqt","key":"9e373e65c58e47cd242b60485f5a9729"}

In [66]:
# Download the dataset Mcdonals
! kaggle datasets download -d nelgiriyewithana/mcdonalds-store-reviews

Dataset URL: https://www.kaggle.com/datasets/nelgiriyewithana/mcdonalds-store-reviews
License(s): other
mcdonalds-store-reviews.zip: Skipping, found more recently modified local copy (use --force to force download)


In [67]:
# Unzip the dataset
with zipfile.ZipFile('/teamspace/studios/this_studio/PROJECT_FINAL/mcdonalds-store-reviews.zip', 'r') as zip_ref:
    zip_ref.extractall('/teamspace/studios/this_studio/PROJECT_FINAL/mcdonalds_reviews')

# Load the CSV file
data = pd.read_csv('/teamspace/studios/this_studio/PROJECT_FINAL/mcdonalds_reviews/McDonald_s_Reviews.csv', encoding='latin1', encoding_errors='replace')

In [68]:
df1 = data.copy()

## Feature engineering

### Feauture engineering on review_time column

In [69]:
# Create function to parse the column review_time with the current date
def parse_relative_date(s, current_date):

    parts = s.strip().split()
    if len(parts) != 3:
        return None

    # This checks if the number is represented as "a" or "an" , as shown the dataset
    # we have columns that says a monthh ago which equivalent to 1 month ago
    num_str = parts[0].lower()
    if num_str in ['a', 'an']:
        num = 1
    else:
        try:
            num = int(num_str)
        except ValueError:
            return None

    unit = parts[1].lower()

    # This adjusts the current date based on the chosen unit
    if 'day' in unit:
        abs_date = current_date - pd.Timedelta(days=num)
    elif 'month' in unit:
        abs_date = current_date - relativedelta(months=num)
    elif 'year' in unit:
        abs_date = current_date - relativedelta(years=num)
    else:
        return None

    # Normalize to remove the time (sets time to 00:00:00)
    return abs_date.normalize()

# Current date timestamp
current_date = pd.Timestamp.now()

# Apply the function to create a new column with the actual dates
df1['review_date'] = df1['review_time'].apply(lambda x: parse_relative_date(x, current_date))

# Drop the column df1['review_time']
df1 = df1.drop(columns='review_time')
df1.head()


,reviewer_id,store_name,category,store_address,latitude,longitude,rating_count,review,rating,review_date
0,1,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",Why does it look like someone spit on my food?...,1 star,2025-01-05
1,2,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",It'd McDonalds. It is what it is as far as the...,4 stars,2025-03-31
2,3,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",Made a mobile order got to the speaker and che...,1 star,2025-03-31
3,4,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",My mc. Crispy chicken sandwich was ï¿½ï¿½ï¿½ï¿...,5 stars,2025-03-05
4,5,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240","I repeat my order 3 times in the drive thru, a...",1 star,2025-02-05


### Feauture engineering on store_address column

In [70]:
df1[['City', 'State']] = df1['store_address'].apply(lambda x: pd.Series(x.split(', ')[-3:-1]))

# Drop the column df1['store_address']
df1 = df1.drop(columns='store_address')
df1.head()

,reviewer_id,store_name,category,latitude,longitude,rating_count,review,rating,review_date,City,State
0,1,McDonald's,Fast food restaurant,30.460718,-97.792874,"1,240",Why does it look like someone spit on my food?...,1 star,2025-01-05,Austin,TX 78750
1,2,McDonald's,Fast food restaurant,30.460718,-97.792874,"1,240",It'd McDonalds. It is what it is as far as the...,4 stars,2025-03-31,Austin,TX 78750
2,3,McDonald's,Fast food restaurant,30.460718,-97.792874,"1,240",Made a mobile order got to the speaker and che...,1 star,2025-03-31,Austin,TX 78750
3,4,McDonald's,Fast food restaurant,30.460718,-97.792874,"1,240",My mc. Crispy chicken sandwich was ï¿½ï¿½ï¿½ï¿...,5 stars,2025-03-05,Austin,TX 78750
4,5,McDonald's,Fast food restaurant,30.460718,-97.792874,"1,240","I repeat my order 3 times in the drive thru, a...",1 star,2025-02-05,Austin,TX 78750


In [71]:
# Remove the 'star' in the column rating 
df1["rating"] = (df1["rating"].str.split(" ").str[0].astype(int))

### Drop unnecessary columns

In [72]:
# Drop columns, 'store_name', 'category','rating_count'
df1 = df1.drop(columns=['store_name', 'category','rating_count'])

In [73]:
df1.columns

Index(['reviewer_id', 'latitude ', 'longitude', 'review', 'rating',
       'review_date', 'City', 'State'],
      dtype='object')

## Check unique values and duplicates

In [74]:
# Check unique values
columns_to_check = ['reviewer_id', 'latitude ', 'longitude', 'review', 'rating',
                    'review_date', 'City', 'State']

unique_counts = df1[columns_to_check].nunique()

print(unique_counts)

reviewer_id    33396
latitude          39
longitude         39
review         22285
rating             5
review_date       29
City              26
State             36
dtype: int64


In [75]:
# Check for duplicates

duplicate_rows = df1[df1.duplicated()]
print(f"Total duplicate rows: {duplicate_rows.shape[0]}")


Total duplicate rows: 0


## Clean column 'review' and remove special characters

In [76]:
df2 = df1.copy()

In [77]:
# Create function to remove special characters in the review column
def clean_review(review):
    #review = review.lower()
    review = review.replace('½ï', '').replace('ý', '')
    review = re.sub(r'[^a-zA-Z\s]', '', review)
    review = re.sub(r'\s+', ' ', review).strip()
    return review

df2['clean_reviews'] = df2['review'].apply(clean_review)

print(df2[['clean_reviews']])

                                           clean_reviews
0      Why does it look like someone spit on my food ...
1      Itd McDonalds It is what it is as far as the f...
2      Made a mobile order got to the speaker and che...
3      My mc Crispy chicken sandwich was customer ser...
4      I repeat my order times in the drive thru and ...
...                                                  ...
33391                         They treated me very badly
33392                           The service is very good
33393                         To remove hunger is enough
33394   Its good but lately it has become very expensive
33395                          they took good care of me

[33396 rows x 1 columns]


## Create new colum 'actual_sentiment' and assign sentiments(negative, positive and neutral) for each rating

In [78]:
# Function to add column for the actual review: positive : ratings > 3,
# neutral: ratings == 3, negative : ratings <3.
def actual_sentiment(data):
    if data['rating'] > 3:
        return "positive" 
    elif data["rating"] == 3:
        return "neutral"
    else:
        return "negative"

# Apply the function to each row
df2['actual_sentiment'] = df2.apply(actual_sentiment, axis=1)

In [79]:
# Drop columns 'review'
df2 = df2.drop(columns='review')

In [80]:
df2.head()

,reviewer_id,latitude,longitude,rating,review_date,City,State,clean_reviews,actual_sentiment
0,1,30.460718,-97.792874,1,2025-01-05,Austin,TX 78750,Why does it look like someone spit on my food ...,negative
1,2,30.460718,-97.792874,4,2025-03-31,Austin,TX 78750,Itd McDonalds It is what it is as far as the f...,positive
2,3,30.460718,-97.792874,1,2025-03-31,Austin,TX 78750,Made a mobile order got to the speaker and che...,negative
3,4,30.460718,-97.792874,5,2025-03-05,Austin,TX 78750,My mc Crispy chicken sandwich was customer ser...,positive
4,5,30.460718,-97.792874,1,2025-02-05,Austin,TX 78750,I repeat my order times in the drive thru and ...,negative


## Check duplicates in the 'clean_review' column

In [81]:
duplicates = df2[df2['clean_reviews'].duplicated()]
print(f"Total duplicate reviews: {duplicates.shape[0]}")
duplicates.head()


Total duplicate reviews: 11581


,reviewer_id,latitude,longitude,rating,review_date,City,State,clean_reviews,actual_sentiment
426,427,30.460718,-97.792874,5,2021-04-05,Austin,TX 78750,Good food and good service,positive
463,464,30.460718,-97.792874,5,2024-04-05,Austin,TX 78750,Good,positive
470,471,30.460718,-97.792874,4,2024-04-05,Austin,TX 78750,Great service,positive
479,480,30.460718,-97.792874,3,2020-04-05,Austin,TX 78750,Fast,neutral
495,496,30.460718,-97.792874,4,2022-04-05,Austin,TX 78750,Great service,positive


In [82]:
# Drop duplicates in the column
df2 = df2.drop_duplicates(subset='clean_reviews')
print(f"New dataset size: {df2.shape[0]}")

New dataset size: 21815


### Save the clean dataset

In [83]:
df2.to_csv('/teamspace/studios/this_studio/PROJECT_FINAL/mcdonalds_reviews/cleaned_data.csv',index=False)